In [1]:
import netCDF4 as nc
import numpy as np

In [2]:
# Copyright 2013-2016 The Salish Sea NEMO Project and
# The University of British Columbia

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#    http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Functions for working with geographical data and model results.
"""
import numpy as np


def distance_along_curve(lons, lats):
    """Calculate cumulative distance in km between points in lons, lats

    :arg lons: 1D array of longitude points.
    :type lons: :py:class:`numpy.ndarray`

    :arg lats: 1D array of latitude points.
    :type lats: :py:class:`numpy.ndarray`

    :returns: Cumulative point-by-point distance along track in km.
    :rtype: :py:class:`numpy.ndarray`
    """
    dist = np.cumsum(haversine(lons[1:], lats[1:], lons[:-1], lats[:-1]))
    return np.insert(dist, 0, 0)


def haversine(lon1, lat1, lon2, lat2):
    """Calculate the great-circle distance in kilometers between two points
    on a sphere from their longitudes and latitudes.

    Reference: http://www.movable-type.co.uk/scripts/latlong.html

    :arg lon1: Longitude of point 1.
    :type lon1: float or :py:class:`numpy.ndarray`

    :arg lat1: Latitude of point 1.
    :type lat1: float or :py:class:`numpy.ndarray`

    :arg lon2: Longitude of point 2.
    :type lon2: float or :py:class:`numpy.ndarray`

    :arg lat2: Latitude of point 2.
    :type lat2: float or :py:class:`numpy.ndarray`

    :returns: Great-circle distance between two points in km
    :rtype: float or :py:class:`numpy.ndarray`
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km


def _spiral_search_for_closest_water_point(
    j, i, land_mask, lon, lat, model_lons, model_lats
):
    # Searches in a spiral pattern around grid element (j,i)
    # for the closest water point to the the coordinate (lat,lon)

    jmax, imax = land_mask.shape
    # Limit on size of grid search
    max_search_dist = max(50, int(model_lats.shape[1]/4))
    closest_point = None
    j_s, i_s = j, i  # starting point is j, i
    dj, di = 0, -1
    # move j_s, i_s in a square spiral centred at j, i
    while (i_s-i) <= max_search_dist:
        if any([(j_s-j) == (i_s-i),
               ((j_s-j) < 0 and (j_s-j) == -(i_s-i)),
               ((j_s-j) > 0 and (j_s-j) == 1-(i_s-i))]):
            # Hit the corner of the spiral- change direction
            dj, di = -di, dj
        i_s, j_s = i_s+di, j_s+dj  # Take a step to next square
        if (i_s >= 0
            and i_s < imax
            and j_s >= 0
            and j_s < jmax
            and not land_mask[j_s, i_s]
        ):
            # Found a water point, how close is it?
            actual_dist = haversine(
                lon, lat, model_lons[j_s, i_s], model_lats[j_s, i_s])
            if closest_point is None:
                min_dist = actual_dist
                closest_point = (j_s, i_s)
            elif actual_dist < min_dist:
                # Keep record of closest point
                min_dist = actual_dist
                closest_point = (j_s, i_s)
            # Assumes grids are square- reduces search radius to only
            # check grids that could potentially be closer than this
            grid_dist = int(((i_s-i)**2 + (j_s-j)**2)**0.5)
            if (grid_dist + 1) < max_search_dist:
                # Reduce stopping distance for spiral-
                # just need to check that no points closer than this one
                max_search_dist = grid_dist + 1
    if closest_point is not None:
        return closest_point
    else:
        raise ValueError('lat/lon on land and no nearby water point found')


def find_closest_model_point(
    lon, lat, model_lons, model_lats, grid='NEMO', land_mask=None,
    tols={
        'NEMO': {'tol_lon': 0.0104, 'tol_lat': 0.00388},
        'GEM2.5': {'tol_lon': 0.016, 'tol_lat': 0.012},
        'WW3': {'tol_lon': 0.001, 'tol_lat': 0.002}
        }
):
    """Returns the grid coordinates of the closest model point
    to a specified lon/lat. If land_mask is provided, returns the closest
    water point.

    Example:

    .. code-block:: python

        j, i = find_closest_model_point(
                   -125.5,49.2,model_lons,model_lats,land_mask=bathy.mask)

    where bathy, model_lons and model_lats are returned from
    :py:func:`salishsea_tools.tidetools.get_bathy_data`.

    j is the y-index(latitude), i is the x-index(longitude)

    :arg float lon: longitude to find closest grid point to

    :arg float lat: latitude to find closest grid point to

    :arg model_lons: specified model longitude grid
    :type model_lons: :py:obj:`numpy.ndarray`

    :arg model_lats: specified model latitude grid
    :type model_lats: :py:obj:`numpy.ndarray`

    :arg grid: specify which default lon/lat tolerances
    :type grid: string

    :arg land_mask: describes which grid coordinates are land
    :type land_mask: numpy array

    :arg tols: stored default tols for different grid types
    :type tols: dict

    :returns: yind, xind
    """

    if grid not in tols:
        raise KeyError(
            'The provided grid type is not in tols. '
            'Use another grid type or add your grid type to tols.')

    # Search for a grid point with longitude and latitude within
    # tolerance of measured location
    j_list, i_list = np.where(
        np.logical_and(
            (np.logical_and(model_lons > lon - tols[grid]['tol_lon'],
                            model_lons < lon + tols[grid]['tol_lon'])),
            (np.logical_and(model_lats > lat - tols[grid]['tol_lat'],
                            model_lats < lat + tols[grid]['tol_lat']))
        )
    )

    if len(j_list) == 0:
        # Added by BMM March 2017
        # If including points outside of domain:
        return np.nan, np.nan
        # raise ValueError(
        #    'No model point found. tol_lon/tol_lat too small or '
        #    'lon/lat outside of domain.')
    try:
        j, i = map(np.asscalar, (j_list, i_list))
    except ValueError:
        # Several points within tolerance
        # Calculate distances for all and choose the closest

        # Avoiding array indexing because some functions
        # pass in model_lons and model_lats as netcdf4 objects
        # (which treat 'model_lons[j_list, i_list]' differently)
        lons = [model_lons[j_list[n], i_list[n]] for n in range(len(j_list))]
        lats = [model_lats[j_list[n], i_list[n]] for n in range(len(j_list))]
        dists = haversine(
            np.array([lon] * i_list.size), np.array([lat] * j_list.size),
            lons, lats)
        n = dists.argmin()
        j, i = map(np.asscalar, (j_list[n], i_list[n]))

    # If point is on land and land mask is provided
    # try to find closest water point
    if land_mask is None or not land_mask[j, i]:
        return j, i
    try:
        return _spiral_search_for_closest_water_point(
            j, i, land_mask, lon, lat, model_lons, model_lats)
    except ValueError:
        raise ValueError(
            'lat/lon on land and no nearby water point found')

In [12]:
PLACES = {
    # Tide gauge stations
    'Campbell River': {
        # deg E, deg N
        'lon lat': (-125.24, 50.04),
        # Canadian Hydrographic Service (CHS) or NOAA
        'stn number': 8074,
        # m above chart datum
        'mean sea lvl': 2.916,
        # m above chart datum
        'hist max sea lvl': 5.35,
        # indices of nearest weather forcing grid point
        # j is the latitude (y) direction, i is the longitude (x) direction
        'wind grid ji': (190, 102),
        # indices of nearest NEMO model grid point
        # j is the latitude (y) direction, i is the longitude (x) direction
        'NEMO grid ji': (747, 125),
    },
    'Cherry Point': {
        'lon lat': (-122.766667, 48.866667),
        'stn number': 9449424,
        'mean sea lvl': 3.543,
        'hist max sea lvl': 5.846,
        'wind grid ji': (122, 166),
        'NEMO grid ji': (343, 341),
    },
    'Nanaimo': {
        'lon lat': (-123.93, 49.16),
        'mean sea lvl': 3.08,
        'hist max sea lvl': 5.47,
        'wind grid ji': (142, 133),
        'NEMO grid ji': (484, 208),
    },
    'Point Atkinson': {
        'lon lat': (-123.25, 49.33),
        'stn number': 7795,
        'mean sea lvl': 3.09,
        'hist max sea lvl': 5.61,
        'wind grid ji': (146, 155),
        'NEMO grid ji': (468, 329),
    },
    'Victoria': {
        'lon lat': (-123.36, 48.41),
        'stn number': 7120,
        'mean sea lvl': 1.8810,
        'hist max sea lvl': 3.76,
        'wind grid ji': (104, 144),
        'NEMO grid ji': (297, 197),
    },

    # Ferry terminals
    'Tsawwassen': {
        'lon lat': (-123.1281, 49.0084),
    },
    'Duke Pt.': {
        'lon lat': (-123.8909, 49.1632),
    },
    'Horseshoe Bay': {
        'lon lat': (-123.2728, 49.3742),
    },
    'Departure Bay': {
        'lon lat': (-123.8909, 49.1632),
    },
    'Swartz Bay': {
        'lon lat': (-123.4102, 48.6882),
    },

    # Cities
    'Vancouver': {
        'lon lat': (-123.1207, 49.2827),
    },

    # Provinces and states
    'British Columbia': {
        'lon lat': (-123.6, 49.9),
    },
    'Washington State': {
        'lon lat': (-123.8, 47.8),
    },

    # Bodies of water
    'Pacific Ocean': {
        'lon lat': (-125.6, 48.1),
    },
    'Juan de Fuca Strait': {
        'lon lat': (-124.7, 48.47),
    },
    'Puget Sound': {
        'lon lat': (-122.67, 48),
    },
    'Strait of Georgia': {
        'lon lat': (-123.8, 49.3),
    },

    # VENUS
    'Central node': {
        # location from Ocean Networks Canada (ONC) website
        'lon lat': (-123.425825, 49.040066666),
        # depth in metres from ONC website
        'depth': 294,
        # NEMO python grid indices: j in y direction, i in x direction
        'NEMO grid ji': (424, 266),
        # ONC data web services API station code
        'ONC stationCode': 'SCVIP',
    },
    'Delta BBL node': {
        # ONC's description is "Delta/Lower Slope/Bottom Boundary Layer"
        'lon lat': (-123.339633, 49.074766),
        'depth': 143,
        'NEMO grid ji': (424, 283),
        'ONC stationCode': 'LSBBL',
    },
    'Delta DDL node': {
        # ONC's description is "Delta/Upper Slope/Delta Dynamics Laboratory"
        'lon lat': (-123.32972, 49.08495),
        'depth': 107,
        'NEMO grid ji': (426, 286),
        'ONC stationCode': 'USDDL',
    },
    'East node': {
        'lon lat': (-123.316836666, 49.04316),
        'depth': 164,
        'NEMO grid ji': (417, 283),
        'ONC stationCode': 'SEVIP',
    },

    # Lightstations
    'Sandheads': {
        'lon lat': (-123.30, 49.10),
        'NEMO grid ji': (426, 292),
    },

    # Wave buoys
    'Halibut Bank': {
        'lon lat': (-123.72, 49.34),
        'NEMO grid ji': (503, 261),
        'GEM2.5 grid ji': (149, 141),
        'EC buoy number': 46146,
    },
    'Sentry Shoal': {
        'lon lat': (-125.0, 49.92),
        'NEMO grid ji': (707, 145),
        'GEM2.5 grid ji': (183, 107),
        'EC buoy number': 46131,
    }
}


#: Names of tide gauge sites,
#: ordered from south and west to north and east.
#: These names are keys of the :py:data:`~salishsea_tools.places.PLACES` dict.
TIDE_GAUGE_SITES = (
    'Victoria', 'Cherry Point', 'Point Atkinson', 'Nanaimo', 'Campbell River',
)

In [5]:
fb=nc.Dataset('/results/SalishSea/wwatch3-forecast/SoG_ww3_fields_20170515_20170517.nc')
nav_lon=np.copy(fb.variables['longitude'])
nav_lat=np.copy(fb.variables['latitude'])
hs = np.copy(fb.variables['hs'])
lm = np.copy(fb.variables['hs'])
MAPSTA = np.copy(fb.variables['MAPSTA'])

fb.close()

In [6]:
lat, lon = np.meshgrid(nav_lat, nav_lon)
flon = lon-360
flat = lat

In [19]:
for i in TIDE_GAUGE_SITES:
    lo, la = find_closest_model_point( PLACES[i]['lon lat'][0], PLACES[i]['lon lat'][1], flon, flat )
    PLACES[i]['ww3 grid ji'] = la, lo
    PLACES[i]['ww3 lon lat'] = nav_lon[ PLACES[i]['ww3 grid ji'][1]] - 360, nav_lat[ PLACES[i]['ww3 grid ji'][0] ]

In [20]:
PLACES

{'British Columbia': {'lon lat': (-123.6, 49.9)},
 'Campbell River': {'NEMO grid ji': (747, 125),
  'hist max sea lvl': 5.35,
  'lon lat': (-125.24, 50.04),
  'mean sea lvl': 2.916,
  'stn number': 8074,
  'wind grid ji': (190, 102),
  'ww3 grid ji': (453, 109),
  'ww3 lon lat': (-125.23699951171875, 50.038502)},
 'Central node': {'NEMO grid ji': (424, 266),
  'ONC stationCode': 'SCVIP',
  'depth': 294,
  'lon lat': (-123.425825, 49.040066666)},
 'Cherry Point': {'NEMO grid ji': (343, 341),
  'hist max sea lvl': 5.846,
  'lon lat': (-122.766667, 48.866667),
  'mean sea lvl': 3.543,
  'stn number': 9449424,
  'wind grid ji': (122, 166),
  'ww3 grid ji': (193, 462),
  'ww3 lon lat': (-122.76600646972656, 48.8685)},
 'Delta BBL node': {'NEMO grid ji': (424, 283),
  'ONC stationCode': 'LSBBL',
  'depth': 143,
  'lon lat': (-123.339633, 49.074766)},
 'Delta DDL node': {'NEMO grid ji': (426, 286),
  'ONC stationCode': 'USDDL',
  'depth': 107,
  'lon lat': (-123.32972, 49.08495)},
 'Departure

In [ ]:
#hs test
for k in TIDE_GAUGE_SITES:
    lon,lat = PLACES[k]['ww3 lon lat'] #lon, lat
    print (lat,lon)
    print (k, hs[10,lat,lon])